# PyMapD and PyGDF Demo on NY Taxi Data Subset

In this example, we will use PyMapD to create and populate a table of NY Taxi data from a CSV file.  Then, we query the MapD database to get a PyGDF GPU dataframe and manipulate the data using groupby, join and other dataframe operations.

In [1]:
import pygdf 
import pymapd

## Connect to MapD

In [2]:
con = pymapd.connect(user="mapd", password="HyperInteractive", host="localhost", dbname="mapd")

### Make table

In [3]:
con.execute("DROP TABLE IF EXISTS nytaxi_subset;")
con.execute("""CREATE TABLE nytaxi_subset (
vendor_id                  TEXT ENCODING DICT,
rate_code                  INTEGER,
store_and_fwd_flag         TEXT ENCODING DICT,
passenger_count            INTEGER,
trip_time_in_secs          INTEGER,
trip_distance              DOUBLE,
pickup_longitude           DOUBLE,
pickup_latitude            DOUBLE,
dropoff_longitude          DOUBLE,
dropoff_latitude           DOUBLE,
tolls_amount               DOUBLE,
tip_amount                 DOUBLE,
total_amount               DOUBLE,
mta_tax                    DOUBLE,
fare_amount                DOUBLE,
payment_type               TEXT ENCODING DICT,
surcharge                  DOUBLE,
pickup_datetime_year       INTEGER,
pickup_datetime_month      INTEGER,
pickup_datetime_day        INTEGER,
pickup_datetime_hours      INTEGER,
dropoff_datetime_year      INTEGER,
dropoff_datetime_month     INTEGER,
dropoff_datetime_day       INTEGER,
dropoff_datetime_hours     INTEGER
);
""")

### Load data from csv

Decompress CSV archive

In [4]:
!gunzip -kf ./data/nytaxi/nytaxi_pre_mapd_200k.csv.gz

Load data

In [5]:
PWD = !pwd
con.execute("COPY nytaxi_subset FROM '{PWD}/data/nytaxi/nytaxi_pre_mapd_200k.csv';".format(PWD=PWD[0]))

## Ingest data from MapD

In [6]:
df = con.select_ipc_gpu("""
SELECT  
payment_type, pickup_longitude, pickup_latitude, tip_amount, total_amount, fare_amount
FROM nytaxi_subset 
""")

Check the type of the result from `con.select_ipc_gpu` is a GPU dataframe

In [7]:
type(df)

pygdf.dataframe.DataFrame

In [8]:
print('nrows', len(df))

nrows 200000


Inspect column types

In [9]:
df.dtypes

payment_type        category
pickup_longitude     float64
pickup_latitude      float64
tip_amount           float64
total_amount         float64
fare_amount          float64
dtype: object

In [10]:
df.head().to_pandas()

,payment_type,pickup_longitude,pickup_latitude,tip_amount,total_amount,fare_amount
0,CSH,-73.987556,40.761391,0.00,4.50,4.0
1,CRD,-73.990776,40.755871,1.38,7.38,5.5
2,CRD,-73.964905,40.763874,1.40,8.90,7.0
3,CRD,-73.989098,40.773476,1.00,7.00,5.5
4,CRD,-73.992874,40.736671,2.50,20.00,17.0


## Groupby lat lon grid

We want to group each record by their pickup location. We will to round the lat lon with the ``round_latlon`` method.  By using ``.applymap``, the rounding method will be compiled into GPU code.

In [11]:
from math import floor

def round_latlon(x):
    scale = 5
    return floor(x * scale) / scale

In [12]:
group_df = df.loc[:, ['pickup_longitude', 'pickup_latitude', 'tip_amount', 'fare_amount']] 

group_df['pickup_longitude'] = group_df['pickup_longitude'].applymap(round_latlon)
group_df['pickup_latitude'] = group_df['pickup_latitude'].applymap(round_latlon)

group_df['tip_ratio'] = group_df['tip_amount'] / group_df['fare_amount']


In [13]:
group_df.dtypes

pickup_longitude    float64
pickup_latitude     float64
tip_amount          float64
fare_amount         float64
tip_ratio           float64
dtype: object

In [14]:
group_df.head().to_pandas()

,pickup_longitude,pickup_latitude,tip_amount,fare_amount,tip_ratio
0,-74.0,40.6,0.00,4.0,0.000000
1,-74.0,40.6,1.38,5.5,0.250909
2,-74.0,40.6,1.40,7.0,0.200000
3,-74.0,40.6,1.00,5.5,0.181818
4,-74.0,40.6,2.50,17.0,0.147059


Here, we run groupby and specify the aggregating methods on each column.

In [15]:
from collections import OrderedDict

# Aggregating methods to apply to each column
aggs = OrderedDict()
aggs['tip_amount'] = 'mean'
aggs['fare_amount'] = ['mean', 'std', 'count']
aggs['tip_ratio'] = 'mean'


grouped_stats = group_df.groupby(['pickup_longitude', 'pickup_latitude']).agg(aggs)
print('total groups', len(grouped_stats))
grouped_stats.head().to_pandas()

total groups 50


,pickup_longitude,pickup_latitude,tip_amount,fare_amount_mean,fare_amount_std,fare_amount_count,tip_ratio
0,-98.2,40.6,0.9,4.50,0.00,1.0,0.200000
1,-94.2,40.6,0.0,30.25,21.75,2.0,0.000000
2,-81.0,40.6,2.4,12.00,0.00,1.0,0.200000
3,-75.4,40.6,2.2,10.50,0.00,1.0,0.209524
4,-74.6,40.6,1.2,6.00,0.00,1.0,0.200000


Reorder the grouped dataframe by `fare_amount_count`

In [16]:
grouped_stats.sort_values('fare_amount_count', ascending=False).head().to_pandas()

,pickup_longitude,pickup_latitude,tip_amount,fare_amount_mean,fare_amount_std,fare_amount_count,tip_ratio
13,-74.0,40.6,1.175462,11.251773,8.682359,162910.0,0.099798
9,-74.2,40.6,1.279872,11.780425,8.612637,24133.0,0.112059
20,-73.8,40.6,4.731888,45.513919,15.343363,5011.0,0.097381
14,-74.0,40.8,1.276300,12.889100,10.017667,4211.0,0.091650
43,0.0,0.0,1.884570,15.715963,19.860135,3582.0,0.161140


## Groupby payment type

We can also group by categorical columns.

In [17]:
group_pay = df.loc[:, ['payment_type', 'tip_amount', 'fare_amount']]
group_pay['tip_ratio'] = group_df['tip_ratio']

groupby_payment = group_pay.groupby(['payment_type']).mean()
groupby_payment.sort_values('tip_ratio', ascending=False).to_pandas()

,payment_type,tip_amount,fare_amount,tip_ratio
2,UNK,3.623360,18.695040,0.205597
0,CRD,2.462649,13.355459,0.194828
1,CSH,0.000118,11.130542,0.000009
3,DIS,0.000000,5.250000,0.000000
4,NOC,0.000000,2.500000,0.000000


## Join table with payment_type meaning

We can use `.join()` to add a description column for each payment type

In [18]:
import pandas
import numpy as np

payment_code = {
    'CRD': 'Credit Card',
    'CSH': 'Cash',
    'NOC': 'No Charge',
    'DIS': 'Dispute',
    'UNK': 'Unknown',
}

payment_meaning = pygdf.DataFrame()

# Customize codes.dtype to match storage type from mapd
src_cat = group_pay.payment_type
cat = pandas.Categorical(payment_code.keys(), categories=src_cat.cat.categories)
payment_meaning['payment_type'] = pygdf.Series.from_categorical(cat, codes=cat.codes.astype(src_cat.data.dtype))

payment_meaning['payment_meaning'] = pandas.Categorical(payment_code.values())
payment_meaning = payment_meaning.set_index('payment_type')

payment_meaning.to_pandas()

,payment_meaning
CRD,Credit Card
CSH,Cash
NOC,No Charge
DIS,Dispute
UNK,Unknown


In [19]:
joined = groupby_payment.set_index('payment_type').join(payment_meaning)
joined.sort_values('tip_ratio', ascending=False).to_pandas()

,tip_amount,fare_amount,tip_ratio,payment_meaning
UNK,3.623360,18.695040,0.205597,Unknown
CRD,2.462649,13.355459,0.194828,Credit Card
CSH,0.000118,11.130542,0.000009,Cash
DIS,0.000000,5.250000,0.000000,Dispute
NOC,0.000000,2.500000,0.000000,No Charge
